In [1]:
from pymongo import MongoClient
import datetime
import pandas as pd
import numpy as np
import itertools
import pymongo
from wordcloud import WordCloud

from collections import Counter
import pickle
from gensim.models.doc2vec import Doc2Vec, LabeledSentence
import matplotlib.pyplot as plt
import seaborn as sns

client = MongoClient("localhost", 27017)
db = client["reddit_polarization"]

In [2]:
class RedditPost(object):
    def __init__(self, db, collections, field, min_words):
        self.db = db
        self.collections = collections
        self.field = field
        self.min_words = min_words
    def __iter__(self):        
        for collection in self.collections:
            corpus = db[collection].find({}, {self.field: 1, "_id": 1})
            for post in corpus:
                text = post[self.field].split()
                tag = str(post["_id"])
                if len(text) >= self.min_words:
                    yield LabeledSentence(text, [tag])

In [3]:
reddit_post = RedditPost(db, ["MensRights", "Feminism"], "tokens", 30)

In [4]:
models = [Doc2Vec(dm=0, dbow_words=1, min_count=5, window=15,
                  size=100, sample=1e-5, iter=10, workers=6), # Distributed BOW
          Doc2Vec(dm=1, dm_mean=1, min_count=5, window=15,
                  size=100, sample=1e-5, iter=10, workers=6)] # Distributed Memory

names = ["Distributed BOW", "Distributed Memory"]
model = models[0]
name = names[0]

In [5]:
print "%s: %s" % (name, str(model))
reddit_post = RedditPost(db, ["MensRights", "Feminism"], "tokens", 30)
model.build_vocab(reddit_post)
reddit_post = RedditPost(db, ["MensRights", "Feminism"], "tokens", 30)
model.train(reddit_post)

Distributed BOW: Doc2Vec(dbow+w,d100,n5,w15,mc5,s1e-05,t6)


345140207

In [6]:
model.save("doc2vec_d100_n5_w15_mc5_s1e_05_t6.model")